# Capstone Project

   #### Applied Data Science Capstone by IBM/Coursera

### Introduction: Business Problem

One of the clients is looking for opening a bookstore in **Sydney, Australia**. As part of this project, we will find a optimal location for the client to open his new store **'ABC Book Store'**.

Since there are lot of schools and bookstore in Sydney, our plan is to find a location where we have less book store in school's vicinity.
Some of the conditions that we will focus on will be:
    1. It should be near a good school with rating above 7 out of 10
    2. There should not be more than 1 bookstore within an area of 2 kilometers
    3. If there is a bank/ATM in the vicinity, it would be a plus

We will use our data science powers to generate a few most promissing neighborhoods based on this criteria. Advantages of each area will then be clearly expressed so that best possible final location can be chosen by stakeholders.


### Data

In order to generate the best location, we would need to find 
- schools in Sydney
- schools average rating
- number of existing books store around the school
- other facilities such as bank/atm

We will look for schools within a radius of 30 Km of Sydney. For the existing bookstore or bank/atm we will search within 2 km of radius from the school.


Let's first import packages and retrieve the latitude and longitude of Sydney, NSW, Australia.

In [2]:
!conda install -c conda-forge folium=0.5.0 --yes
!conda install -c conda-forge geopy --yes 

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.0.1               |             py_0         575 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.0 MB

The following NEW packages will be 

In [3]:
import pandas as pd
import numpy as np 
import requests
from geopy.geocoders import Nominatim 
from pandas.io.json import json_normalize
import folium 
import matplotlib.cm as cm
import matplotlib.colors as colors

In [4]:
# Lets get the latitude and longitude of Sydney
city_address = 'Sydney, NSW, Australia'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(city_address)
city_latitude = location.latitude
city_longitude = location.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(city_address, city_latitude, city_longitude))

The geograpical coordinate of Sydney, NSW, Australia are -33.8679574, 151.210047.



Now we have retrieved the coordinated of Sydney Australia. Let's try to generate a list of schools within 30 km of this location.

We will use Foursquare API to get a list of all the schools within Sydney area.


In [5]:
# The code was removed by Watson Studio for sharing.

In [7]:
# Locate all the schools in Sydney
radius = 30000  # Schools within 30 km radius of Sydney
LIMIT = 400
query = 'School'

# create the API request URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&query={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    city_latitude, 
    city_longitude, 
    radius, 
    LIMIT, 
    query)

In [8]:
result = requests.get(url).json()
# print(result['response'].keys())
# print('****************************************************')
# result['response']

In [9]:
# Normalize the result set

st = json_normalize(result)
st = json_normalize(st['response.groups'][0])
st = json_normalize(st['items'][0])
schools_df = st[['venue.id', 'venue.name', 'venue.location.lat', 'venue.location.lng']]
schools_df.rename(columns={'venue.id':'venue_id', 'venue.name':'venue_name', 'venue.location.lat':'venue_location_lat', 'venue.location.lng':'venue_location_lng'}, inplace=True)
schools_df.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


,venue_id,venue_name,venue_location_lat,venue_location_lng
0,4baddb2af964a520b36d3be3,Alliance Française de Sydney,-33.871547,151.205645
1,4de6d301b0fb9a99f6f41d44,Sydney Business School,-33.861970,151.210330
2,51dbdf44498ed4bfbd977d58,La Lingua,-33.876605,151.208084
3,4b56b555f964a520281928e3,SCEGGS,-33.876418,151.217994
4,4be3467dd27a20a1c6bf915b,Crown Street Public School,-33.882362,151.215050


In [10]:
# Get the shape of the df
schools_df.shape

(100, 4)

Well, there are 100 schools in Sydney. Let's try to get their rating now.

For retrieving rating, we will use Foursquare API.

In [11]:
# Add category
schools_df['category_type'] = 'School'

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [12]:
# create a DF for storing rating so that we dont execute API for which we already have rating

df_columns = ['venue_id', 'rating']
rating_df = pd.DataFrame(columns=df_columns)
rating_df

,venue_id,rating


In [13]:
def get_schools_ratings(venue_ids):
    
    venue_rating = []
    for venue_id in venue_ids:
        # print(venue_id)

        # process if venue_id is not present in rating df
        if len(rating_df[rating_df['venue_id'] == venue_id]) == 0: 
        
            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/{}?&client_id={}&client_secret={}&v={}'.format(
                venue_id,
                CLIENT_ID, 
                CLIENT_SECRET, 
                VERSION)

            # make the GET request
            results = requests.get(url).json()

            try:
                rating = results['response']['venue']['rating']
            except:
                rating = -1

            # append row in rating df
            rating_df.loc[len(rating_df)] = [venue_id, rating]

            # print(results)
        

In [14]:
# get ratings for all the schools

get_schools_ratings(schools_df['venue_id'])

In [15]:
print(rating_df.head(3))
rating_df[rating_df['rating']>7]  


                   venue_id rating
0  4baddb2af964a520b36d3be3     -1
1  4de6d301b0fb9a99f6f41d44     -1
2  51dbdf44498ed4bfbd977d58     -1


,venue_id,rating
37,4e7d218a5503e732bccd6619,8.9


In [17]:
rating_df[rating_df['rating']>0]

# Only one school in sydney is rated on foursquare, so this is not a good criteria
# Assuming, once we have filtered it, we will look for other conditions to fulfill.

,venue_id,rating
37,4e7d218a5503e732bccd6619,8.9


We have hit our first bummer, there is only one school that is having rating out of 100 schools.
Looking at other sources, only few schools has been rated on them as well. So, for now, we will 
not filter any schools and assume all schools are having good rating.

Let's find out the existing books store near each school within a radius of 2 km.

In [18]:
# Let's find out the Bookstore near each school

# nearby_location_info
columns = ['selection_venue_id', 'school_id', 'venue_name', 'latitude', 'longitude', 'category_type', 'sub_category']
nearby_venues = pd.DataFrame(columns=columns)

In [19]:
# Locate all the bookstores and ATM/banks
def get_atm_bookstore_details(venue_ids, lat, lng):
    
    radius = 2000  # Within 2 km radius of location
    LIMIT = 100

    for venue_id, latitude, longitude in zip(venue_ids, lat, lng):
        
        # process if venue_id is not present in rating df
        query = 'Bookstore'
        if len(nearby_venues[(nearby_venues['school_id'] == venue_id) & (nearby_venues['category_type'] == query)]) == 0: 
        
            # create the API request URL for bookstore
            url_book = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&query={}'.format(
                CLIENT_ID, 
                CLIENT_SECRET, 
                VERSION, 
                latitude, 
                longitude, 
                radius, 
                LIMIT, 
                query)
            
            result = requests.get(url_book).json()
            results = result['response']['groups'][0]['items']
            
            if len(results) > 0:
                for v in results:
                    nearby_venues.loc[len(nearby_venues)] = [        
                        v['venue']['id'],
                        venue_id,
                        v['venue']['name'], 
                        v['venue']['location']['lat'], 
                        v['venue']['location']['lng'], 
                        query,
                        v['venue']['categories'][0]['name']]
        
        # create the API request URL for ATM
        query = 'ATM'
        if len(nearby_venues[(nearby_venues['school_id'] == venue_id) & (nearby_venues['category_type'] == query)]) == 0:
            
            url_atm = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&query={}'.format(
                CLIENT_ID, 
                CLIENT_SECRET, 
                VERSION, 
                latitude, 
                longitude, 
                radius, 
                LIMIT, 
                query)
            
            result = requests.get(url_atm).json()
            results = result['response']['groups'][0]['items']

            if len(results) > 0:
                for v in results:
                    nearby_venues.loc[len(nearby_venues)] = [        
                        v['venue']['id'],
                        venue_id,
                        v['venue']['name'], 
                        v['venue']['location']['lat'], 
                        v['venue']['location']['lng'], 
                        query,
                        v['venue']['categories'][0]['name']]


In [20]:
# Get the deatils for all atm/bookstore near the school
get_atm_bookstore_details(schools_df['venue_id'], schools_df['venue_location_lat'], schools_df['venue_location_lng'])

In [21]:
nearby_venues.head(3)

,selection_venue_id,school_id,venue_name,latitude,longitude,category_type,sub_category
0,4b05876bf964a520b09122e3,4baddb2af964a520b36d3be3,Kinokuniya,-33.872456,151.207525,Bookstore,Bookstore
1,4b05876bf964a520af9122e3,4baddb2af964a520b36d3be3,Dymocks Sydney,-33.869796,151.207101,Bookstore,Bookstore
2,4b60d991f964a5208bfd29e3,4baddb2af964a520b36d3be3,Abbey's Bookshop,-33.872169,151.206682,Bookstore,Bookstore


In [37]:
# Let's find out the number of bookstore that are within 2km radius of each school
print('Number of Bookstore: ',nearby_venues[nearby_venues['category_type'] == 'Bookstore'].shape)
print('Number of ATM/Banks: ',nearby_venues[nearby_venues['category_type'] == 'ATM'].shape)

Number of Bookstore:  (772, 8)
Number of ATM/Banks:  (368, 8)


Wow! There are 772 bookstores and 368 banks/ATMs near these 100 schools and that too within 2 km radius of each school.

In [22]:
def set_category(category):
    return 1 if category == 'Bookstore' else 0

In [23]:
# Set a category for each type of venue (atm/bookstore)
nearby_venues['cat'] = nearby_venues['category_type'].apply(set_category)

In [24]:
nearby_venues.head(2)

,selection_venue_id,school_id,venue_name,latitude,longitude,category_type,sub_category,cat
0,4b05876bf964a520b09122e3,4baddb2af964a520b36d3be3,Kinokuniya,-33.872456,151.207525,Bookstore,Bookstore,1
1,4b05876bf964a520af9122e3,4baddb2af964a520b36d3be3,Dymocks Sydney,-33.869796,151.207101,Bookstore,Bookstore,1


Let's visualize all the schools, bookstores and atm/banks we retrieved.

In [26]:
# Let's visualize it on the map (All Schools/ATMs/Bookstores)

# create map
map_clusters = folium.Map(location=[city_latitude, city_longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(len(nearby_venues))
ys = [i + x + (i*x)**2 for i in range(len(nearby_venues))]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add Scheel markers to the map
markers_colors = []
for lat, lon, school_name in zip(schools_df['venue_location_lat'], schools_df['venue_location_lng'], schools_df['venue_name']):
    label = folium.Popup(f'{school_name}', parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='green',
        fill_opacity=0.7).add_to(map_clusters)
       
# add Bookstore and ATM markers to the map
markers_colors = []
for lat, lon, category, cat in zip(nearby_venues['latitude'], nearby_venues['longitude'], nearby_venues['category_type'], nearby_venues['cat']):
    label = folium.Popup(f'{category}', parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cat],
        fill=True,
        fill_color=rainbow[cat],
        fill_opacity=0.7).add_to(map_clusters)
    
map_clusters

In [27]:
# Let's check the number of bookstore and ATMs near each schools

school_summary = nearby_venues.pivot_table(values='selection_venue_id', index='school_id', columns='category_type', aggfunc='count')
school_summary.reset_index(inplace=True)

In [28]:
school_summary.sort_values('Bookstore', ascending=False, inplace=True)
school_summary['Bookstore'] = school_summary['Bookstore'].astype('int')
school_summary['ATM'].fillna(0, inplace=True)
school_summary['ATM'] = school_summary['ATM'].astype('int')

In [29]:
school_details = pd.merge(school_summary, schools_df, how='left', left_on='school_id', right_on='venue_id')
school_details_top_30 = school_details[school_details['Bookstore'] > 30]
school_details_top_30.head(2)

,school_id,ATM,Bookstore,venue_id,venue_name,venue_location_lat,venue_location_lng,category_type
0,51dbdf44498ed4bfbd977d58,12,35,51dbdf44498ed4bfbd977d58,La Lingua,-33.876605,151.208084,School
1,4ba00f3ef964a520845637e3,12,35,4ba00f3ef964a520845637e3,St Andrews Cathedral School,-33.873796,151.205836,School


Oh God, can you believe this, there are **35 book stores** close to two schools that too within 2 km radius. It's going to be a pretty tough competition there.

Let's see where these schools are located on the map 

In [30]:
# Let's visualize schools with maximum number of bookstores:

# create map
map_clusters = folium.Map(location=[city_latitude, city_longitude], zoom_start=14)

# set color scheme for the clusters
x = np.arange(len(school_details_top_30))
ys = [i + x + (i*x)**2 for i in range(len(school_details_top_30))]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
rainbow = ['#ff0000', '#ffff00', '#40ff00', '#0000ff', '#ff00ff', '#ff0000', '#00ffff', '#bf00ff']

# add Scheel markers to the map
markers_colors = []
for lat, lon, school_name, atm, bookstore in zip(school_details_top_30['venue_location_lat'], school_details_top_30['venue_location_lng'], 
                                                 school_details_top_30['venue_name'], school_details_top_30['ATM'], school_details_top_30['Bookstore']):
    label = folium.Popup(f'{school_name}, ATM: {atm}, Bookstore: {bookstore}', parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[bookstore-30],
        fill=True,
        fill_color=rainbow[bookstore-30],
        fill_opacity=0.9).add_to(map_clusters)
    
map_clusters

In [34]:
# Well the central of city is covered with hell lot of stores
# Let's find the schools with least number of bookstores

school_top_option = school_details[(school_details['Bookstore'] < 2) & (school_details['ATM'] > 0 )]
print('shape:',school_top_option.shape)
school_top_option.head(10)

shape: (7, 8)


,school_id,ATM,Bookstore,venue_id,venue_name,venue_location_lat,venue_location_lng,category_type
83,4bc3b5e52a89ef3b8476f588,1,1,4bc3b5e52a89ef3b8476f588,Loreto Normanhurst,-33.726886,151.097341,School
85,4bc15f154cdfc9b6e2ac9421,1,1,4bc15f154cdfc9b6e2ac9421,Kambala Girls School,-33.864954,151.271718,School
87,4c6b9a09a48420a1af660a0b,1,1,4c6b9a09a48420a1af660a0b,Peakhurst Public School,-33.956055,151.061306,School
88,4d75cc3d48b7f04d7dc4ecf5,2,1,4d75cc3d48b7f04d7dc4ecf5,Leichhardt Public School,-33.884161,151.158287,School
90,4c6f032dd274b60c1054d50d,2,1,4c6f032dd274b60c1054d50d,Mortdale Public School,-33.972588,151.081708,School
91,4b9ad808f964a52076db35e3,1,1,4b9ad808f964a52076db35e3,Canterbury Ice Rink,-33.909925,151.113154,School
94,4bda02e5c79cc9282b0a7de9,1,1,4bda02e5c79cc9282b0a7de9,Kincoppal-Rose Bay,-33.863426,151.271848,School


Finally, we have found 7 schools around which we have only one book store and have atleast 1 atm/bank within schools 2 km radius.

Let's see where these schools are located on the map

In [32]:
# Let's visualize schools with minimum number of bookstores:

# create map
map_clusters = folium.Map(location=[city_latitude, city_longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(len(school_top_option))
ys = [i + x + (i*x)**2 for i in range(len(school_top_option))]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
rainbow = ['#00ffff','#0000ff','#ff0000','#ff00ff',   '#ffff00', '#40ff00',  '#ff0000', '#bf00ff']

# add Scheel markers to the map
markers_colors = []
for lat, lon, school_name, atm, bookstore in zip(school_top_option['venue_location_lat'], school_top_option['venue_location_lng'], 
                                                 school_top_option['venue_name'], school_top_option['ATM'], school_top_option['Bookstore']):
    label = folium.Popup(f'{school_name}, ATM: {atm}, Bookstore: {bookstore}', parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[atm],
        fill=True,
        fill_color=rainbow[atm],
        fill_opacity=0.9).add_to(map_clusters)
    
map_clusters


### Results and Discussion

Our results show that although we had 100 schools with around 800 books store and approximately 400 atm's/banks within a radius of 2 km from the schools, there are few schools where number of books store was minimal. There were few schools around which staggering number of books stores existed (35 book stores). Most of these schools were located in Central/Financial district of Sydney. Our potential location candidate for book stores were in the suburbs of Sydney.

With narrowing down all the available schools and as per the requirement, there were 7 schools that have at least 1 atm/bank and at max 1 book store within its vicinity. These schools are located in:
1. one school in upper north suburb of Sydney
2. two schools in western suburb of Sydney
3. two schools in south-west suburb of Sydney and 
4. two schools in eastern suburb of Sydney


### Conclusion

Purpose of this project was to identify an optimal location for opening a book store with least number of bookstores around a school and have at least one bank or atm near it. Identifying these schools, existing bank stores and banks or ATM's according with the help for Foursquare API's has helped achieve our requirement by exploration of these datapoints and can be shared with the stakeholders.

Final decision on optimal restaurant location will be made by stakeholders based on specific characteristics of neighbourhoods and locations in every recommended zone, taking into consideration additional factors like attractiveness of each location (proximity to park or water), levels of noise / proximity to major roads, real estate availability, prices, social and economic dynamics of every neighbourhood etc.
